In [3]:
# 필요한 라이브러리 재로드 및 데이터 재처리
import pandas as pd
from scipy.stats import ttest_ind, chi2_contingency
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from datetime import datetime
import nbformat as nbf

In [4]:
# 데이터 로드
df = pd.read_csv("D:/김동영/11_Github/mygit-1/DAT/25-1프로젝트/최종데이터/최종데이터.csv")

In [5]:
df

,Username,follower,following,City,Country,median_views,median_comments,average_views,average_comments,last_video_date,verified,top_hashtags,진행상황,total_likes,total_videos
0,theonlyhaven,1200000,77,Singapore,SG,17.6K,8.0,134.6K,32.0,2019-12-19,False,"rlholiday, poloralphlauren, rlholidaytree, ral...",1,60.0M,1.8K
1,turquoise.dec,755000,122,Singapore,SG,117.8K,34.0,341.6K,153.0,2021-02-14,False,"yslbeauty, yslbeautysg, yslmakeup, ysltheinks",1,33.1M,188
2,srsaskin_,1439,890,Birmingham,GB,1.1K,19.0,2.7K,28.0,2024-02-03,False,"kbeauty, koreanskincare, teca, fyp, abib, abib...",0,17.1K,52
3,_anjello,949,681,Concord,US,389,24.0,529,27.0,2024-01-14,False,"skincare, capcut, kbeauty, koreanskincare, ski...",0,8.6K,153
4,jf.skin24,896,841,Quezon city,PH,484,20.0,561,22.0,2024-01-24,False,"jfskindiary, kbeauty, skintok, koreanskincare,...",0,6.1K,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128,sya.skin,7433,441,Siem Reap,Cambodia,10.5K,15.0,30.4K,21.0,2022-03-18,False,"stylekorean, stylekoreanbox, skincare, styleko...",1,110.6K,137
129,emileerachel,2119,4891,Jupiter,US,348,2.0,1.1K,3.0,2023-08-11,False,"benefitcosmetics, mominfluencer, stopmotion, a...",1,6.9K,278
130,jhamheartsthis,2878,599,Tabaco,PH,798,10.0,8.0K,15.0,2021-09-09,False,"blush, kbeauty, fypシ?, cutemakeup, viralmakeup...",0,164.1K,298
131,aglimpseofglow,60,82,La Habra,US,948,0.0,952,1.0,2022-12-26,False,"ugccontentcreators, skincareinfluencer, makeup...",0,2.6K,55


In [6]:
# 수치형 변환 함수
def parse_number(x):
    try:
        if isinstance(x, str):
            x = x.strip().upper().replace(",", "")
            if 'K' in x:
                return float(x.replace('K', '')) * 1_000
            elif 'M' in x:
                return float(x.replace('M', '')) * 1_000_000
            else:
                return float(x)
        return x
    except:
        return None

# 변환 적용
columns_to_convert = ['median_views', 'average_views', 'total_likes', 'total_videos']
for col in columns_to_convert:
    df[col] = df[col].apply(parse_number)

In [7]:
# 파생 변수 생성
df['likes_per_follower'] = df['total_likes'] / df['follower']
df['videos_per_follower'] = df['total_videos'] / df['follower']
df['last_video_date'] = pd.to_datetime(df['last_video_date'], errors='coerce')
reference_date = pd.to_datetime("2024-05-29")
df['days_since_last_video'] = (reference_date - df['last_video_date']).dt.days
df['videos_per_day'] = df['total_videos'] / df['days_since_last_video']

In [8]:
# 해시태그 유사도 계산
df['top_hashtags'] = df['top_hashtags'].fillna('')
# 비교할 제품 키워드(문자열)를 정의
reference_keywords = "kbeauty cosmetic cleansing beauty"
# TF-IDF 벡터라이저 객체 생성
vectorizer = TfidfVectorizer()

# 데이터프레임의 top_hashtags와 reference_keywords를 합쳐서 벡터화
# 마지막 행이 reference_keywords에 해당합니다.
tfidf_matrix = vectorizer.fit_transform(df['top_hashtags'].tolist() + [reference_keywords])

# 해시태그 벡터(전체 데이터)와 기준 키워드 벡터(reference_keywords)를 분리
hashtag_vectors = tfidf_matrix[:-1] # 데이터프레임의 각 row에 해당하는 해시태그 벡터
reference_vector = tfidf_matrix[-1] # 기준 키워드 벡터

# 각 row의 해시태그 벡터와 기준 키워드 벡터 간의 코사인 유사도를 계산
cosine_similarities = cosine_similarity(hashtag_vectors, reference_vector.reshape(1, -1)).flatten()

# 계산된 유사도를 데이터프레임에 새로운 컬럼으로 추가
df['hashtag_similarity'] = cosine_similarities

In [9]:
# 그룹 분리
group_0 = df[df['진행상황'] == 0]
group_1 = df[df['진행상황'] == 1]

# 카이제곱 검정
country_crosstab = pd.crosstab(df['Country'], df['진행상황'])
chi2_stat, chi2_p, _, _ = chi2_contingency(country_crosstab)

# t-test 결과 수집
results = {
    "변수명": [],
    "이행 그룹 평균": [],
    "불이행 그룹 평균": [],
    "p-value": [],
    "유의수준 0.05 하에서 유의함": []
}

variables = [
    'likes_per_follower', 'videos_per_follower', 'average_views', 'average_comments',
    'median_views', 'median_comments', 'videos_per_day', 'hashtag_similarity'
]

for var in variables:
    g0 = group_0[var].dropna()
    g1 = group_1[var].dropna()
    pval = ttest_ind(g0, g1, equal_var=False).pvalue
    results["변수명"].append(var)
    results["이행 그룹 평균"].append(g0.mean())
    results["불이행 그룹 평균"].append(g1.mean())
    results["p-value"].append(pval)
    results["유의수준 0.05 하에서 유의함"].append(pval < 0.05)

# 카이제곱 결과 추가
results["변수명"].append("Country (카이제곱)")
results["이행 그룹 평균"].append(None)
results["불이행 그룹 평균"].append(None)
results["p-value"].append(chi2_p)
results["유의수준 0.05 하에서 유의함"].append(chi2_p < 0.05)

# 결과 데이터프레임 생성
results_df = pd.DataFrame(results).round(4)

In [10]:
# 결과 출력
results_df

,변수명,이행 그룹 평균,불이행 그룹 평균,p-value,유의수준 0.05 하에서 유의함
0,likes_per_follower,39.0894,32.7246,0.5153,False
1,videos_per_follower,0.1890,0.0889,0.0021,True
2,average_views,21707.5455,52166.3214,0.0947,False
3,average_comments,13.6623,32.1964,0.0063,True
4,median_views,1152.1818,9141.1429,0.0131,True
5,median_comments,9.7273,14.1250,0.1341,False
6,videos_per_day,1.0565,0.8168,0.3454,False
7,hashtag_similarity,0.0255,0.0150,0.0304,True
8,Country (카이제곱),NaN,NaN,0.1073,False


In [11]:
import numpy as np
from scipy.stats import t

P-value 값을 가지고 유의성 검정을 하기에는 한계가 존재한다. 그렇기 때문에 추가적인 통계 분석 방법을 적용해 함께 고려하는 것이 중요하다.   
수치형 독리변수인 경우에는 Cohen's d 지표를 사용해 P-value 값을 보완한다.   
Cohen's d는 종속변수가 이진 분류일 때 각 그룹에서 독립변수(연속형 변수)의 평균 차이가 얼마나 큰 지를 보여주는 효과 크기 지표이다.   
즉, Cohen's d가 클수록 두 그룹(종속변수 값이 0인 그룹 vs 1인 그룹)에서 해당 독립변수의 평균 차이가 크다는 뜻이다.   
실무에서는 t-test의 p-value와 함께 Cohen's d를 같이 해석한다.   
예) 유의미한 차이가 있을 때, 그 차이가 얼마나 큰 지를 Cohen's d 지표를 사용해 확인한다.   

In [12]:
# 추가적인 통계 분석 방법 예시

# 1. 효과 크기(Effect Size) 계산: Cohen's d (두 그룹 평균 차이의 표준화)
def cohens_d(x, y):
  nx, ny = len(x), len(y)
  pooled_std = (((nx - 1) * x.std() ** 2 + (ny - 1) * y.std() ** 2) / (nx + ny - 2)) ** 0.5
  return (x.mean() - y.mean()) / pooled_std

effect_sizes = {}
for var in variables:
  g0 = group_0[var].dropna()
  g1 = group_1[var].dropna()
  if len(g0) > 1 and len(g1) > 1:
    effect_sizes[var] = cohens_d(g0, g1)
  else:
    effect_sizes[var] = None

print("Cohen's d (Effect Size) for each variable:")
for var, d in effect_sizes.items():
  print(f"{var}: {d}")

# # 2. 신뢰구간(Confidence Interval) 계산: 평균 차이의 95% 신뢰구간

# def mean_diff_ci(x, y, alpha=0.05):
#   nx, ny = len(x), len(y)
#   mean_diff = x.mean() - y.mean()
#   se = np.sqrt(x.var(ddof=1)/nx + y.var(ddof=1)/ny)
#   dof = min(nx, ny) - 1
#   t_crit = t.ppf(1 - alpha/2, dof)
#   ci = (mean_diff - t_crit * se, mean_diff + t_crit * se)
#   return ci

# print("\n95% 신뢰구간 (평균 차이):")
# for var in variables:
#   g0 = group_0[var].dropna()
#   g1 = group_1[var].dropna()
#   if len(g0) > 1 and len(g1) > 1:
#     ci = mean_diff_ci(g0, g1)
#     print(f"{var}: {ci}")
#   else:
#     print(f"{var}: 데이터 부족")

# # 3. 상관분석: 피어슨 상관계수
# print("\n피어슨 상관계수 (진행상황 vs 연속형 변수):")
# for var in variables:
#   corr = df[['진행상황', var]].corr().iloc[0, 1]
#   print(f"{var}: {corr}")

Cohen's d (Effect Size) for each variable:
likes_per_follower: 0.10407279630746667
videos_per_follower: 0.4984523995365743
average_views: -0.30485199010456593
average_comments: -0.5677550726889447
median_views: -0.5278769958556436
median_comments: -0.2846272040335195
videos_per_day: 0.15564138922958262
hashtag_similarity: 0.3707057065725723


In [17]:
# 종속변수(진행상황) 0과 1을 바꿔줌
df['진행상황'] = df['진행상황'].apply(lambda x: 1 if x == 0 else 0)
y = df['진행상황']

In [18]:
import statsmodels.api as sm

X = df[['videos_per_follower', 'hashtag_similarity']].copy()
X = sm.add_constant(X)
y = df['진행상황']

model = sm.Logit(y, X).fit()
print(model.summary())

Optimization terminated successfully.
         Current function value: 0.631113
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                   진행상황   No. Observations:                  133
Model:                          Logit   Df Residuals:                      130
Method:                           MLE   Df Model:                            2
Date:                Wed, 04 Jun 2025   Pseudo R-squ.:                 0.07275
Time:                        12:35:40   Log-Likelihood:                -83.938
converged:                       True   LL-Null:                       -90.524
Covariance Type:            nonrobust   LLR p-value:                  0.001380
                          coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.3612      0.267     -1.354      0.176      -0.884       0.162
vi

In [19]:
print(df[['진행상황', 'videos_per_follower', 'hashtag_similarity']].corr())

                         진행상황  videos_per_follower  hashtag_similarity
진행상황                 1.000000             0.240682            0.181361
videos_per_follower  0.240682             1.000000            0.087449
hashtag_similarity   0.181361             0.087449            1.000000
